In [ ]:
import time
import json

import datadog_api_client
from datadog_api_client.v1.api import metrics_api

from datetime import datetime, timedelta
from dateutil import tz

import os

In [ ]:
configuration = datadog_api_client.v1.Configuration()

configuration.api_key['apiKeyAuth'] = os.environ['datadog_api_key_auth']
configuration.api_key['appKeyAuth'] = os.environ['datadog_app_key_auth']

In [ ]:
api_client = datadog_api_client.v1.ApiClient(configuration)

In [ ]:
api = metrics_api.MetricsApi(api_client)

In [ ]:
request_from = int((datetime.now(tz=tz.gettz('UTC')) - timedelta(days=2)).timestamp())
request_to   = int(datetime.now(tz=tz.gettz('UTC')).timestamp())

In [ ]:
request_to

In [ ]:
request_from = datetime(2021, 10, 30, 6, 29, tzinfo=tz.gettz('UTC')).timestamp()
request_to   = int(datetime(2021, 11,  3, 6, 29, tzinfo=tz.gettz('UTC')).timestamp())

In [ ]:
result_true = \
    api \
        .query_metrics(
            request_from,
            request_to,
            ' '
        )

In [ ]:
result_false = \
    api \
        .query_metrics(
            request_from,
            request_to,
            'autosmooth(sum:feature_flag_check{enabled:false,feature:improved_outcomes_management,environment:prod} by {feature}.as_count())'
        )

In [ ]:
result_true_dict  = result_true.to_dict()
result_false_dict = result_false.to_dict()

In [ ]:
points_true  = result_true_dict ['series'][0]['pointlist']
points_false = result_false_dict['series'][0]['pointlist']

In [ ]:
{datetime.fromtimestamp(points_true[i][0] / 1000, tz.gettz('UTC')): points_true[i][1] / (points_true[i][1] + points_false[i][1]) * 100 for i in range(len(points_true))}